# Installing Necessary Libraries 

In [ ]:
# !pip install PyMuPDF
# !pip install spacy
# !pip install stix2
# !pip install bertopic
# !pip install sentence-transformers

In [ ]:
!pip install umap-learn

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
spacy.cli.download("en_core_web_sm")

# Importing Libraries 

In [1]:
import pymupdf as fitz
import re
from stix2 import Indicator, AttackPattern, Relationship, Report, Bundle
import json
from datetime import datetime
import TTPelement
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from umap.umap_ import UMAP
from bertopic import BERTopic
from hdbscan import HDBSCAN
from bertopic.representation import KeyBERTInspired, PartOfSpeech, MaximalMarginalRelevance
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import pandas as pd
import torch
import pickle
import numpy as np
import os
import spacy
from transformers import DistilBertTokenizer, AutoTokenizer
from transformers import DistilBertForSequenceClassification, AutoModelForSequenceClassification
import logging
import sys
import time

In [2]:
nlp = spacy.load("en_core_web_sm")

# PDF text Extraction

In [3]:
def extract_text_from_pdf(pdf_path):
    with fitz.open(pdf_path) as doc:
        text = ""
        for page in doc:
            text += page.get_text()
    return text

In [ ]:
text=extract_text_from_pdf('aptreport.pdf')

In [ ]:
text

# Text Cleaning and Chunking

In [4]:
def CleanText(text):
  cleaned_text = re.sub(r'\s+', ' ', text)    # remove blank spaces
  cleaned_text = re.sub(r'[^a-zA-Z0-9\s.,!?:/()\[\]@_-]+', '', cleaned_text)     # keep only required characters

  return cleaned_text
    
def chunk_sentences(text):
    """Chunk the cleaned text into sentences using spaCy."""
    doc = nlp(text)  # Process the text with spaCy
    sentences = [sent.text.strip() for sent in doc.sents]  # Extract sentences
    return [s for s in sentences if s]  # Filter out empty strings

In [ ]:
cleaned_text = CleanText(text)
clean=chunk_sentences(cleaned_text)

In [ ]:
clean

# Topic Modelling

In [5]:
class StemTokenizer:
    def __init__(self):
        self.stemmer = PorterStemmer()

    def __call__(self, doc):
        return [self.stemmer.stem(t) for t in word_tokenize(doc)]

def create_pos_patterns():
    """Define POS patterns for the PartOfSpeech representation model."""
    return [
        [{'POS': 'ADJ'}, {'POS': 'NOUN'}],  # e.g., "malicious software"
        [{'POS': 'NOUN'}],                  # e.g., "malware", "ransomware"
        [{'POS': 'VERB'}, {'POS': 'NOUN'}], # e.g., "exploit vulnerability"
        [{'POS': 'NOUN'}, {'POS': 'NOUN'}], # e.g., "threat actor", "data breach"
        [{'POS': 'PROPN'}],                 # e.g., Specific group names like "APT29"
        [{'POS': 'NOUN'}, {'POS': 'PROPN'}] # e.g., "attacks by Lazarus Group"
    ]

def build_representation_model():
    """Build the representation model for BERTopic."""
    pos_patterns = create_pos_patterns()
    return {
        "Main": KeyBERTInspired(),
        "POS": [
            PartOfSpeech("en_core_web_sm", pos_patterns=pos_patterns),
            MaximalMarginalRelevance(diversity=0.4)
        ],
    }

def create_vectorizer():
    """Create a custom CountVectorizer with a stemming tokenizer."""
    return CountVectorizer(
        min_df=1,         # Capture rare words
        max_df=0.95,      # Avoid overly common words
        ngram_range=(1, 2),
        stop_words='english',
        tokenizer=StemTokenizer()
    )

def create_umap_model():
    """Create a UMAP model with adjusted parameters for smaller datasets."""
    return UMAP(
        n_neighbors=3,
        n_components=5,
        min_dist=0.0,
        metric='cosine',
        random_state=42
    )

def create_hdbscan_model():
    """Create an HDBSCAN model with adjusted parameters for smaller datasets."""
    return HDBSCAN(
        min_cluster_size=2,
        min_samples=1,
        metric='euclidean',
        cluster_selection_method='eom',
        prediction_data=True
    )

def build_topic_model(clean_data):
    """
    Build and fit a BERTopic model using the custom components.

    Parameters:
        clean_data (list of str): The preprocessed and cleaned dataset.

    Returns:
        tuple: The fitted BERTopic model, topics, and initial probabilities.
    """
    representation_model = build_representation_model()
    vectorizer_model = create_vectorizer()
    umap_model = create_umap_model()
    hdbscan_model = create_hdbscan_model()

    topic_model = BERTopic(
        nr_topics=None,  # Adjust or set to None for automatic determination
        vectorizer_model=vectorizer_model,
        representation_model=representation_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        verbose=True
    )

    # Fit the model
    topic_model.fit(clean_data)

    # Transform the data
    topics, ini_probs = topic_model.transform(clean_data)
    return topic_model, topics, ini_probs

In [ ]:
# Build and fit the BERTopic model
topic_model, topics, ini_probs = build_topic_model(clean)

In [ ]:
topics = topic_model.get_representative_docs()

In [ ]:
topics

In [ ]:
for topic_id, docs in topics.items():
    print(f"Topic {topic_id}:")
    for doc in docs:
        print(f"- {doc}")
    print("\n")

# IOC Extraction and Soft Tagging

In [6]:
def get_ttp_elements_from_chunks(chunks):
    elements = []
    counts = []
    replaced = []

    for chunk in chunks:
        elems, count, replaced_values = TTPelement.GetTTPelements(chunk)
        elements.append(elems)
        counts.append(count)
        replaced.append(replaced_values)
        
#     print("Elements:", elements)
#     print("Counts:", counts)
#     print("Replaced Values:", replaced)

    return elements, counts, replaced

In [ ]:
chunks=get_ttp_elements_from_chunks(topics)

In [7]:
def process_dict_with_ttp(data):
    processed_results = {}  # Dictionary to store processed results

    for key, sentences in data.items():
        # Ensure sentences is a list; skip if not
        if not isinstance(sentences, list):
            raise ValueError(f"Expected a list of sentences for key '{key}', got {type(sentences)}")

        # Merge the sentences into a single string for each key
        merged_sentences = " ".join(sentences)  # Join with a space or use ". " if you want a period between sentences

        # Process the merged sentences using get_ttp_elements_from_chunks
        elements, counts, replaced = get_ttp_elements_from_chunks([merged_sentences])  # Pass the merged sentence list

        # Structure the processed results
        processed_results[key] = {
            "Original Sentences": sentences,
            "Merged Sentences": merged_sentences,
            "Elements": elements,
            "Counts": counts,
            "Replaced Sentences": replaced,
        }

    return processed_results

In [ ]:
# Process the dictionary
results = process_dict_with_ttp(topics)

# Display results for each key
for key, result in results.items():
    print(f"Results for Key {key}:\n")
    for i, sentence in enumerate(result["Original Sentences"]):
        print(f"Sentence: {sentence}")
        print(f"  Elements: {result['Elements'][i]}")
        print(f"  Word Count: {result['Counts'][i]}")
        print(f"  Replaced Sentence: {result['Replaced Sentences'][i]}")
        print()


# Model

In [8]:
# Load the model, tokenizer, and multilabel binarizer
model_dir = "distilbert-finedtune"

In [9]:
# Load the model
model = DistilBertForSequenceClassification.from_pretrained(model_dir)
model.eval()  # Set to evaluation mode

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [10]:
# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained(model_dir)

In [11]:
# Load the multilabel binarizer
with open("multilabel_binarizer.pkl", "rb") as f:
    multilabel_binarizer = pickle.load(f)

In [12]:
# Load the CSV file to create a label-to-column mapping
csv_path = "Labels.csv"
label_data = pd.read_csv(csv_path)

In [13]:
label_data = label_data.drop(['Unnamed: 0'],axis=1)

In [14]:
# Normalize columns in the CSV
label_data['Tactic-Name'] = label_data['Tactic-Name'].str.strip().str.lower()
label_data['Technique-Name'] = label_data['Technique-Name'].str.strip().str.lower()
label_data['SubTechnique-Name'] = label_data['SubTechnique-Name'].str.strip().str.lower()
label_data['id'] = label_data['id'].str.strip().str.lower()

In [15]:
# Create a label-to-column mapping
tactic_labels = label_data['Tactic-Name'].unique().tolist()
technique_labels = label_data['Technique-Name'].unique().tolist()
sub_technique_labels = label_data['SubTechnique-Name'].unique().tolist()
technique_id_labels = label_data['id'].unique().tolist()

In [16]:
def categorize_label(label):
    label = label.strip().lower()
    if label in tactic_labels:
        return "Tactic Name"
    elif label in technique_labels:
        return "Technique Name"
    elif label in sub_technique_labels:
        return "Sub-Technique Name"
    elif label in technique_id_labels:
        return "Technique ID"
    else:
        return "Unknown"

In [17]:
def analyze_output(structured_output):
    # Normalize all values in structured_output for consistency
    structured_output = {key: [val.strip().lower() for val in value] for key, value in structured_output.items()}
    
    # Check if all four fields are given and valid
    if all(len(structured_output[key]) > 0 for key in structured_output):
        matching_rows = label_data[(label_data['Tactic-Name'].isin(structured_output['Tactic Name'])) &
                                   (label_data['Technique-Name'].isin(structured_output['Technique Name'])) &
                                   (label_data['SubTechnique-Name'].isin(structured_output['Sub-Technique Name'])) &
                                   (label_data['id'].isin(structured_output['Technique ID']))]
        if not matching_rows.empty:
            # print(f"[DEBUG] All four fields match:\n{matching_rows}\n")
            return structured_output
    
    # Handle cases with three fields given
    if len(structured_output['Tactic Name']) > 0 and len(structured_output['Technique Name']) > 0 and len(structured_output['Sub-Technique Name']) > 0:
        matching_rows = label_data[(label_data['Tactic-Name'].isin(structured_output['Tactic Name'])) &
                                   (label_data['Technique-Name'].isin(structured_output['Technique Name'])) &
                                   (label_data['SubTechnique-Name'].isin(structured_output['Sub-Technique Name']))]
        if not matching_rows.empty:
            return find_missing_labels_from_rows(structured_output, matching_rows)

    if len(structured_output['Tactic Name']) > 0 and len(structured_output['Technique Name']) > 0 and len(structured_output['Technique ID']) > 0:
        matching_rows = label_data[(label_data['Tactic-Name'].isin(structured_output['Tactic Name'])) &
                                   (label_data['Technique-Name'].isin(structured_output['Technique Name'])) &
                                   (label_data['id'].isin(structured_output['Technique ID']))]
        if not matching_rows.empty:
            return find_missing_labels_from_rows(structured_output, matching_rows)

    if len(structured_output['Tactic Name']) > 0 and len(structured_output['Sub-Technique Name']) > 0 and len(structured_output['Technique ID']) > 0:
        matching_rows = label_data[(label_data['Tactic-Name'].isin(structured_output['Tactic Name'])) &
                                   (label_data['SubTechnique-Name'].isin(structured_output['Sub-Technique Name'])) &
                                   (label_data['id'].isin(structured_output['Technique ID']))]
        if not matching_rows.empty:
            return find_missing_labels_from_rows(structured_output, matching_rows)

    if len(structured_output['Technique Name']) > 0 and len(structured_output['Sub-Technique Name']) > 0 and len(structured_output['Technique ID']) > 0:
        matching_rows = label_data[(label_data['Technique-Name'].isin(structured_output['Technique Name'])) &
                                   (label_data['SubTechnique-Name'].isin(structured_output['Sub-Technique Name'])) &
                                   (label_data['id'].isin(structured_output['Technique ID']))]
        if not matching_rows.empty:
            return find_missing_labels_from_rows(structured_output, matching_rows)

    # Handle cases with two fields given
    valid_pairs = []
    matching_rows_dict = []
    keys = list(structured_output.keys())
    for i in range(len(keys)):
        for j in range(len(keys)):
            if i != j:
                key1, key2 = keys[i], keys[j]
                values1, values2 = structured_output[key1], structured_output[key2]
                for value1 in values1:
                    for value2 in values2:
                        if key1 == "Tactic Name" and key2 == "Technique Name":
                            matching_rows = label_data[(label_data['Tactic-Name'] == value1) &
                                                       (label_data['Technique-Name'] == value2)]
                        elif key1 == "Tactic Name" and key2 == "Sub-Technique Name":
                            matching_rows = label_data[(label_data['Tactic-Name'] == value1) &
                                                       (label_data['SubTechnique-Name'] == value2)]
                        elif key1 == "Tactic Name" and key2 == "Technique ID":
                            matching_rows = label_data[(label_data['Tactic-Name'] == value1) &
                                                       (label_data['id'] == value2)]
                        elif key1 == "Technique Name" and key2 == "Sub-Technique Name":
                            matching_rows = label_data[(label_data['Technique-Name'] == value1) &
                                                       (label_data['SubTechnique-Name'] == value2)]
                        elif key1 == "Technique Name" and key2 == "Technique ID":
                            matching_rows = label_data[(label_data['Technique-Name'] == value1) &
                                                       (label_data['id'] == value2)]
                        elif key1 == "Sub-Technique Name" and key2 == "Technique ID":
                            matching_rows = label_data[(label_data['SubTechnique-Name'] == value1) &
                                                       (label_data['id'] == value2)]
                        else:
                            matching_rows = pd.DataFrame()
                        
                        if not matching_rows.empty:
                            valid_pairs.append((key1, value1, key2, value2))
                            matching_rows_dict.append(matching_rows)

    # Debug: Print valid pairs formed
    # print(f"[DEBUG] Valid pairs formed: {valid_pairs}\n")
    
    # Create new outputs for each valid pair and find missing labels
    merged_output = {"Tactic Name": [], "Technique Name": [], "Sub-Technique Name": [], "Technique ID": []}
    for idx, pair in enumerate(valid_pairs):
        key1, value1, key2, value2 = pair
        matching_rows = matching_rows_dict[idx]
        temp_output = {"Tactic Name": [], "Technique Name": [], "Sub-Technique Name": [], "Technique ID": []}
        temp_output[key1].append(value1)
        temp_output[key2].append(value2)
        completed_output = find_missing_labels_from_rows(temp_output, matching_rows)
        for key in merged_output:
            merged_output[key].extend([val for val in completed_output[key] if val not in merged_output[key]])
    
    # Handle cases with a single field given
    for key in structured_output:
        if len(structured_output[key]) > 0:
            column_name = key.replace(' ', '-')
            if column_name in label_data.columns:
                matching_rows = label_data[label_data[column_name].isin(structured_output[key])]
                if not matching_rows.empty:
                    completed_output = find_missing_labels_from_rows(structured_output, matching_rows)
                    for key in merged_output:
                        merged_output[key].extend([val for val in completed_output[key] if val not in merged_output[key]])
    
    return merged_output

In [18]:
def find_missing_labels_from_rows(filtered_output, matching_rows):
    # Normalize all values in filtered_output for consistency
    filtered_output = {key: [val.strip().lower() for val in value] for key, value in filtered_output.items()}

    # Filter rows that match all non-empty fields in filtered_output
    condition = pd.Series(True, index=matching_rows.index)
    for key in filtered_output:
        if filtered_output[key]:
            column_name = None
            if key == "Tactic Name":
                column_name = 'Tactic-Name'
            elif key == "Technique Name":
                column_name = 'Technique-Name'
            elif key == "Sub-Technique Name":
                column_name = 'SubTechnique-Name'
            elif key == "Technique ID":
                column_name = 'id'

            if column_name:
                condition &= matching_rows[column_name].isin(filtered_output[key])

    filtered_matching_rows = matching_rows[condition]

    # Debug: Print filtered matching rows
    # print(f"[DEBUG] Filtered matching rows based on existing output:\n{filtered_matching_rows}\n")

    # Find missing labels based on the filtered output using filtered matching rows
    if not filtered_matching_rows.empty:
        for _, row in filtered_matching_rows.iterrows():
            if len(filtered_output["Tactic Name"]) > 0 and len(filtered_output["Technique Name"]) > 0:
                if len(filtered_output["Sub-Technique Name"]) == 0 and pd.notna(row['SubTechnique-Name']):
                    filtered_output["Sub-Technique Name"].append(row['SubTechnique-Name'])
                if len(filtered_output["Technique ID"]) == 0 and pd.notna(row['id']):
                    filtered_output["Technique ID"].append(row['id'])

            if len(filtered_output["Tactic Name"]) > 0 and len(filtered_output["Sub-Technique Name"]) > 0:
                if len(filtered_output["Technique Name"]) == 0 and pd.notna(row['Technique-Name']):
                    filtered_output["Technique Name"].append(row['Technique-Name'])
                if len(filtered_output["Technique ID"]) == 0 and pd.notna(row['id']):
                    filtered_output["Technique ID"].append(row['id'])

            if len(filtered_output["Technique Name"]) > 0 and len(filtered_output["Sub-Technique Name"]) > 0:
                if len(filtered_output["Tactic Name"]) == 0 and pd.notna(row['Tactic-Name']):
                    filtered_output["Tactic Name"].append(row['Tactic-Name'])
                if len(filtered_output["Technique ID"]) == 0 and pd.notna(row['id']):
                    filtered_output["Technique ID"].append(row['id'])

    return filtered_output

In [19]:
def predict(text):   
    # Tokenize the input text
    encoding = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    
    # Move to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    encoding = {key: value.to(device) for key, value in encoding.items()}

    # Perform inference
    with torch.no_grad():
        outputs = model(**encoding)
        logits = outputs.logits

    # Apply sigmoid to get probabilities
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits).cpu().numpy()

    # Set threshold and get predictions
    threshold = 0.3
    preds = (probs >= threshold).astype(int)

    # Transform predictions back to original labels
    labels = multilabel_binarizer.inverse_transform(preds)

    #Debug: Print the model output
    # print(f"[DEBUG] Model output labels: {labels}\n")

    # Prepare structured output
    structured_output = {"Tactic Name": [], "Technique Name": [], "Sub-Technique Name": [], "Technique ID": []}
    for label_set in labels:
        for label in label_set:
            category = categorize_label(label)
            if category != "Unknown":
                structured_output[category].append(label)
    
    # Analyze the output to filter valid pairs and find missing labels
    final_output = analyze_output(structured_output)

    # Append the original text as a chunk in the final output
    final_output["Chunk"] = [text]

    return final_output

In [ ]:
# Example usage
if __name__ == "__main__":
    test_text = "Phishing impersonating governmental entities as the spreading method. In some campaigns, particularly those involving financial attacks, the group impersonates banking institutions."
    predictions = predict(test_text)
    for label_type, value in predictions.items():
        print(f"{label_type}: {', '.join(value)}")

In [ ]:
# Example usage
if __name__ == "__main__":
    test_text = "Sophisticated cyberattacks frequently involve phishing campaigns, where attackers utilize malicious URLs and email addresses to deceive victims."
    predictions = predict(test_text)
    for label_type, value in predictions.items():
        print(f"{label_type}: {', '.join(value)}")

In [ ]:
predictions

# Final stix bundles)


In [20]:
def CreateStixIndicator(IOCtype, IOCvalue):
    # Define a mapping for IOC types to their STIX patterns
    ioc_mapping = {
        "ipv4": "[ipv4-addr:value = '{}']",
        "ipv6": "[ipv6-addr:value = '{}']",
        "asn": "[autonomous-system:number = '{}']",
        "domain": "[domain-name:value = '{}']",
        "email": "[email-addr:value = '{}']",
        "filename": "[file:name = '{}']",
        "url": "[url:value = '{}']",
        "hash": "[file:hash = '{}']",
        "filepath": "[file:file_path = '{}']",
        "cve": "[vulnerability:cve = '{}']",
        "regkey": "[windows-registry-key:key = '{}']",
        "encodeencryptalgorithms": "[crypto-algorithm:name = '{}']",
        "communicationprotocol": "[network-traffic:protocols[*] = '{}']",
        "dataobject": "[artifact:payload_bin = '{}']"
    }

    # Ensure the IOC type is valid
    if IOCtype not in ioc_mapping:
        raise ValueError(f"Unsupported IOC type: {IOCtype}")

    # Replace backslashes in IOCvalue and format the pattern
    pattern_val = ioc_mapping[IOCtype].format(IOCvalue.replace('\\', '/'))

    # Create the STIX Indicator
    indicator = Indicator(
        name=IOCtype,
        description=f"Extracted {IOCtype}",
        indicator_types=["malicious-activity"],
        pattern=pattern_val,
        labels=["malicious"],
        pattern_type="stix"
    )

    return indicator

In [21]:
def return_stix_objects(counts, tactic_name, technique_name, sub_tech_name, technique_id, iocs, chunks, do_include_non_ioc=False):
    logging.basicConfig(level=logging.INFO)

    indicators_stix = []
    relationships_stix = []
    attack_patterns_stix = []

    for i in range(len(counts)):
        # Use provided values or fallback to defaults
        tactic = tactic_name[i] if i < len(tactic_name) and tactic_name[i] else "Unknown Tactic"
        technique = technique_name[i] if i < len(technique_name) and technique_name[i] else "Unknown Technique"
        sub_tech = sub_tech_name[i] if i < len(sub_tech_name) and sub_tech_name[i] else "No Sub-Technique"
        tech_id = technique_id[i] if i < len(technique_id) and technique_id[i] else "TXXXX"
        ioc = iocs[i] if i < len(iocs) and iocs[i] else {}
        description = chunks[i] if i < len(chunks) and chunks[i] else "No description available."

        # Normalize tactic names (handle both single and multiple tactics)
        tactic_names = tactic if isinstance(tactic, list) else [tactic]

        for tactic in tactic_names:
            # Create the AttackPattern STIX object
            phase_name = tactic.lower().replace(' ', '-').replace('_', '-')
            attack_pattern = AttackPattern(
                name=f"{technique} - {sub_tech}",
                description=description,
                external_references=[{
                    "source_name": "mitre-attack",
                    "external_id": tech_id
                }],
                kill_chain_phases=[{
                    "kill_chain_name": "mitre-attack",
                    "phase_name": phase_name
                }],
                aliases=[technique, sub_tech]
            )
            attack_patterns_stix.append(attack_pattern)

            # Handle IOCs if present
            if ioc:
                for ioc_type, ioc_list in ioc.items():
                    for ioc_value in ioc_list:
                        try:
                            indicator = CreateStixIndicator(ioc_type, str(ioc_value))
                            indicators_stix.append(indicator)

                            # Create relationship between Indicator and AttackPattern
                            relationship = Relationship(
                                source_ref=indicator.id,
                                target_ref=attack_pattern.id,
                                relationship_type="indicates",
                                description=f"Indicator ({ioc_value}) is associated with Attack Pattern ({technique} - {sub_tech}).",
                                start_time=datetime.now(),
                                allow_custom=True
                            )
                            relationships_stix.append(relationship)
                        except Exception as e:
                            logging.error(f"Error creating indicator for IOC ({ioc_value}): {e}")
            elif do_include_non_ioc:
                # Include a "No IOC available" relationship if configured
                logging.info(f"No IOCs found for chunk {i}. Creating placeholder relationship.")
                relationship = Relationship(
                    source_ref=None,
                    target_ref=attack_pattern.id,
                    relationship_type="indicates",
                    description="No IOC available.",
                    start_time=datetime.now(),
                    allow_custom=True
                )
                relationships_stix.append(relationship)

    return indicators_stix, relationships_stix, attack_patterns_stix

In [22]:
def make_stix(indicators, relationships, attack_patterns, clean_text, report_name, labels=None, published=None):
    logging.basicConfig(level=logging.INFO)

    # Default values for optional parameters
    labels = labels or []
    labels.append("threat-report")
    published = published or datetime.now().strftime('%Y-%m-%dT%H:%M:%S.%fZ')

    try:
        # Validate inputs
        if not all(isinstance(obj_list, (list, tuple)) for obj_list in [indicators, relationships, attack_patterns]):
            raise ValueError("Indicators, relationships, and attack patterns must be iterable (e.g., list or tuple).")

        # Collect all object IDs
        all_ids = [obj.id for obj in indicators + relationships + attack_patterns]

        # Log a warning if no objects exist
        if not all_ids:
            logging.warning(f"No STIX objects to include in the report '{report_name}'. Creating an empty bundle.")
            return Bundle(objects=[])

        # Create the report
        report = Report(
            name=report_name,
            description=clean_text,
            published=published,
            object_refs=all_ids,
            labels=labels,
        )

        # Collect all objects into a bundle
        objects = indicators + attack_patterns + relationships + [report]
        bundle = Bundle(objects=objects)

        return bundle

    except Exception as e:
        logging.error(f"An error occurred while creating the STIX bundle for report '{report_name}': {e}")
        return Bundle(objects=[])

# Main Workflow Code

In [ ]:
chunks = [
    """
    The APT group launched a watering hole attack targeting specific industry websites to deliver malware. The IP address 45.33.32.156 was 
    identified as hosting the malicious content, alongside the domain compromised-site.com. Visitors were redirected to this domain and 
    prompted to download a fake software update named UpdateInstaller.exe, which contained malware. The malware would establish persistence 
    by creating registry keys and communicating with the C2 server at 203.0.113.60 using HTTPS.
    """,

    """
    In an escalation of their tactics, the same APT group deployed ransomware via malvertising on popular websites. Users who clicked on the ad were redirected to a download page 
    hosted at malicious-ads.io, where the ransomware executable ransom_payload_v3.2.exe was served. Once executed, the ransomware 
    encrypted files on the victim’s machine and displayed a ransom note directing the victim to contact the attacker via email at 
    support@ransom.com. During encryption, the ransomware communicated with IP addresses like 185.60.218.35 and 192.168.0.1, sending 
    encrypted session keys and victim data.
    """,

    """
    The group also launched brute-force attacks targeting SSH servers within specific critical infrastructure organizations. IP addresses such as 103.21.244.1 and 102.51.145.2 were detected 
    repeatedly attempting login attempts. Once access was obtained, the attackers deployed a reverse shell to 198.51.100.50 over port 443, 
    allowing them to control the compromised system. Further reconnaissance commands were executed to gather system information, which was 
    then exfiltrated to the command and control server. The tactic of initial access through brute force and persistence through remote 
    access tools was identified.
    """,

    """
    Phishing emails were distributed with the subject line “Account Security Notice” to lure users into clicking a malicious link. The emails, 
    sent from noreply@security-alerts.com, claimed to be from the user's bank and included a link to verify account details. The link directed 
    users to http://phishing-site.com/login, a site that mimicked the bank's login page. This page harvested login credentials, which were then 
    used in unauthorized access attempts on financial systems. This is another technique in the APT group's playbook, showing tactics of credential access through phishing and credential dumping.
    """,

    """
    A supply chain attack was discovered, revealing that the APT group compromised a legitimate software vendor’s update server. They injected malware 
    into the update package hosted on update.vendor-software.com. End users who downloaded the update unknowingly installed the malware, which 
    created a backdoor to a remote IP address 203.0.113.45. This backdoor allowed attackers to execute arbitrary commands and exfiltrate data 
    from infected systems. The tactics of lateral movement and exfiltration through command and control were observed.
    """,

    """
    Cybercriminals within the same APT group also targeted e-commerce platforms with SQL injection attacks, attempting to extract credit card details from databases. The 
    attackers operated from IP addresses 198.51.100.88 and 102.51.67.30. Upon successful extraction, the data was encrypted and exfiltrated to 
    a remote server at 203.0.113.77. The exfiltration utilized SSL to secure the data transfer, making detection more challenging. Techniques 
    of data exfiltration and defense evasion were identified in this attack.
    """,

    """
    The group distributed a banking trojan through spam campaigns that claimed to be from a well-known financial institution. The email urged recipients 
    to download a “security patch” from http://safe-banking-update.com/patch.exe, which was actually the trojan. Once installed, the trojan 
    collected browser session data, including saved passwords and autofill information, and exfiltrated this data to the server at 45.33.32.111. 
    The tactics of credential access and data collection were associated with this campaign.
    """,

    """
    Finally, a botnet campaign linked to this APT group was identified, leveraging IoT devices for launching DDoS attacks against critical services. The devices connected to IP addresses 198.51.100.20 and 
    192.0.2.12, forming a network of compromised systems. The botnet periodically sent commands to these IPs to execute coordinated DDoS attacks 
    against high-profile targets. The tactic of impact through service disruption was noted, along with persistence through IoT device compromise.
    """
]

# Main

In [23]:
def main_workflow(pdf_path, base_output_folder):
    logging.basicConfig(level=logging.INFO)
    logging.info(f"Processing {pdf_path} and saving results in {base_output_folder}")

    # Step 0: Extract and Clean Text
    logging.info("Step 0: Extracting text from PDF...")
    raw_text = extract_text_from_pdf(pdf_path)

    logging.info("Step 0: Cleaning the extracted text...")
    clean_text = CleanText(raw_text)

    # Step 1: Chunk the cleaned text into sentences
    logging.info("Step 1: Chunking the cleaned text into sentences...")
    chunks = chunk_sentences(clean_text)

    # Step 2: Build a Topic Model
    logging.info("Step 2: Building the topic model...")
    topic_model, topics, ini_probs = build_topic_model(chunks)

    # Step 3: Fetch representative documents
    logging.info("Step 3: Fetching representative documents for each topic...")
    representative_docs = topic_model.get_representative_docs()

    # Step 4: Process topics using IoC extraction
    logging.info("Step 4: Processing topics for IoC extraction...")
    processed_topics = process_dict_with_ttp(representative_docs)

    # Step 5: Process each topic and generate reports
    logging.info("Step 5: Generating reports...")
    for topic_id, result in processed_topics.items():
        topic_sentences = result["Original Sentences"]
        elements_list = result["Elements"]
        counts_list = result["Counts"]
        replaced_chunks = result["Replaced Sentences"]

        # Create parent folder for the topic
        topic_folder = os.path.join(base_output_folder, f"topic_{topic_id}")
        os.makedirs(topic_folder, exist_ok=True)

        for idx, replaced_chunk in enumerate(replaced_chunks):
            chunk_folder = os.path.join(topic_folder, f"chunk_{idx + 1}")
            os.makedirs(chunk_folder, exist_ok=True)

            logging.info(f"Processing chunk {idx + 1}/{len(replaced_chunks)} of topic {topic_id}...")

            # Step 6: Predict TTPs on the merged chunk
            predictions = predict(replaced_chunk)

            # If predictions are empty, process each sentence separately
            if not predictions:
                logging.warning(f"No predictions available for chunk {idx + 1} of topic {topic_id}. Processing each sentence separately.")
                
                # Split the merged chunk into individual sentences and process each separately
                for sentence in replaced_chunk.split(". "):  # Adjust split based on how sentences are merged
                    sentence = sentence.strip()  # Clean up any extra spaces
                    if sentence:
                        logging.info(f"Processing sentence: {sentence}")
                        sentence_predictions = predict(sentence)  # Predict on individual sentence

                        # Handle predictions for the individual sentence
                        if sentence_predictions:
                            tactic_names = sentence_predictions.get('Tactic Name', ["Unknown"])
                            technique_names = sentence_predictions.get('Technique Name', ["Unknown"])
                            sub_tech_names = sentence_predictions.get('Sub-Technique Name', ["Unknown"])
                            technique_ids = sentence_predictions.get('Technique ID', ["Unknown"])

                            # Ensure consistent list lengths for predictions
                            max_len = max(len(tactic_names), len(technique_names), len(sub_tech_names), len(technique_ids))
                            tactic_names = (tactic_names or ["Unknown"]) * max_len
                            technique_names = (technique_names or ["Unknown"]) * max_len
                            sub_tech_names = (sub_tech_names or ["Unknown"]) * max_len
                            technique_ids = (technique_ids or ["Unknown"]) * max_len

                            # Step 7: Generate STIX Objects for the individual sentence
                            iocs = [elements_list[idx]]
                            counts_list_for_stix = [counts_list[idx]]
                            chunks_for_stix = [sentence]
                            do_include_non_ioc = False

                            for i, (tactic, technique, sub_technique, tech_id) in enumerate(zip(tactic_names, technique_names, sub_tech_names, technique_ids)):
                                try:
                                    indicators, relationships, attack_patterns = return_stix_objects(
                                        counts_list_for_stix,
                                        [tactic],
                                        [technique],
                                        [sub_technique],
                                        [tech_id],
                                        iocs,
                                        chunks_for_stix,
                                        do_include_non_ioc
                                    )

                                    # Skip if no objects were created
                                    if not (indicators or relationships or attack_patterns):
                                        logging.warning(f"No STIX objects created for sentence {sentence}. Skipping.")
                                        continue

                                    # Create STIX bundle
                                    report_name = f"prediction_{i + 1}"
                                    bundle = make_stix(indicators, relationships, attack_patterns, sentence, report_name)

                                    if bundle is None:
                                        logging.warning(f"STIX bundle creation failed for sentence {sentence}, prediction {i + 1}.")
                                        continue

                                    # Save the STIX bundle
                                    stix_file_path = os.path.join(chunk_folder, f"{report_name}.json")
                                    with open(stix_file_path, "w") as f:
                                        json.dump(json.loads(bundle.serialize()), f, indent=4)

                                    logging.info(f"Saved STIX file: {stix_file_path}")

                                except Exception as e:
                                    logging.error(f"Error processing sentence: {sentence}, prediction {i + 1}: {e}")

                        else:
                            logging.warning(f"No predictions available for individual sentence: {sentence}. Skipping.")
                continue  # Skip to next chunk if we handle sentence-by-sentence

            else:
                # Handle predictions for the merged chunk
                tactic_names = predictions.get('Tactic Name', ["Unknown"])
                technique_names = predictions.get('Technique Name', ["Unknown"])
                sub_tech_names = predictions.get('Sub-Technique Name', ["Unknown"])
                technique_ids = predictions.get('Technique ID', ["Unknown"])

                # Ensure consistent list lengths for predictions
                max_len = max(len(tactic_names), len(technique_names), len(sub_tech_names), len(technique_ids))
                tactic_names = (tactic_names or ["Unknown"]) * max_len
                technique_names = (technique_names or ["Unknown"]) * max_len
                sub_tech_names = (sub_tech_names or ["Unknown"]) * max_len
                technique_ids = (technique_ids or ["Unknown"]) * max_len

                # Step 7: Generate STIX Objects
                iocs = [elements_list[idx]]
                counts_list_for_stix = [counts_list[idx]]
                chunks_for_stix = [replaced_chunk]
                do_include_non_ioc = False

                for i, (tactic, technique, sub_technique, tech_id) in enumerate(zip(tactic_names, technique_names, sub_tech_names, technique_ids)):
                    try:
                        indicators, relationships, attack_patterns = return_stix_objects(
                            counts_list_for_stix,
                            [tactic],
                            [technique],
                            [sub_technique],
                            [tech_id],
                            iocs,
                            chunks_for_stix,
                            do_include_non_ioc
                        )

                        # Skip if no objects were created
                        if not (indicators or relationships or attack_patterns):
                            logging.warning(f"No STIX objects created for chunk {idx + 1}, prediction {i + 1}. Skipping.")
                            continue

                        # Create STIX bundle
                        report_name = f"prediction_{i + 1}"
                        bundle = make_stix(indicators, relationships, attack_patterns, replaced_chunk, report_name)

                        if bundle is None:
                            logging.warning(f"STIX bundle creation failed for chunk {idx + 1}, prediction {i + 1}.")
                            continue

                        # Save the STIX bundle
                        stix_file_path = os.path.join(chunk_folder, f"{report_name}.json")
                        with open(stix_file_path, "w") as f:
                            json.dump(json.loads(bundle.serialize()), f, indent=4)

                        logging.info(f"Saved STIX file: {stix_file_path}")

                    except Exception as e:
                        logging.error(f"Error processing chunk {idx + 1}, prediction {i + 1}: {e}")

    logging.info("Workflow completed successfully!")

In [ ]:
# Run main workflow
if __name__ == "__main__":
     main_workflow("Reports/(3) Kimsuky is targeting an arms manufacturer in Europe. _ LinkedIn.pdf", 'testresults')

In [24]:
def process_reports(reports_folder, results_base_folder):
    # Configure logging
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        handlers=[
            logging.StreamHandler(sys.stdout),  # Log to console
            logging.FileHandler("processing.log", mode="w"),  # Log to file
        ],
    )

    # Create the base results folder if it doesn't exist
    os.makedirs(results_base_folder, exist_ok=True)

    logging.info(f"Starting PDF processing from folder: {reports_folder}")
    logging.info(f"Results will be saved in: {results_base_folder}\n")

    processed_files = 0
    skipped_files = 0
    errored_files = []

    # Iterate through each file in the Reports folder
    for report_file in os.listdir(reports_folder):
        if report_file.endswith(".pdf"):  # Process only PDF files
            input_path = os.path.join(reports_folder, report_file)
            report_name = os.path.splitext(report_file)[0]  # Remove the file extension
            output_folder = os.path.join(results_base_folder, report_name)

            # Create a folder for each report's results
            os.makedirs(output_folder, exist_ok=True)

            logging.info(f"Processing file: {report_file}")
            logging.info(f"Input Path: {input_path}")
            logging.info(f"Output Folder: {output_folder}\n")

            try:
                # Call the main workflow for each PDF
                main_workflow(input_path, output_folder)
                processed_files += 1
                logging.info(f"Finished processing: {report_file}\n")
            except Exception as e:
                errored_files.append(report_file)
                logging.error(f"Error processing file {report_file}: {e}\n")

        else:
            skipped_files += 1
            logging.warning(f"Skipping non-PDF file: {report_file}")

    # Summary
    logging.info("PDF processing completed.")
    logging.info(f"Processed files: {processed_files}")
    logging.info(f"Skipped files: {skipped_files}")
    if errored_files:
        logging.error(f"Errored files: {len(errored_files)}")
        for file in errored_files:
            logging.error(f" - {file}")

    logging.info("All tasks have been completed.\n")

In [25]:
def process_reports(reports_folder, results_base_folder):
    # Configure logging
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        handlers=[
            logging.StreamHandler(sys.stdout),  # Log to console
            logging.FileHandler("processing.log", mode="w"),  # Log to file
        ],
    )

    # Create the base results folder if it doesn't exist
    os.makedirs(results_base_folder, exist_ok=True)

    logging.info(f"Starting PDF processing from folder: {reports_folder}")
    logging.info(f"Results will be saved in: {results_base_folder}\n")

    processed_files = 0
    skipped_files = 0
    errored_files = []
    total_files = len([f for f in os.listdir(reports_folder) if f.endswith(".pdf")])

    # Start time tracking
    start_time = time.time()

    # Iterate through each file in the Reports folder
    for report_file in os.listdir(reports_folder):
        if report_file.endswith(".pdf"):  # Process only PDF files
            if processed_files >= 100:  # Stop after processing 100 files
                break

            input_path = os.path.join(reports_folder, report_file)
            report_name = os.path.splitext(report_file)[0]  # Remove the file extension
            output_folder = os.path.join(results_base_folder, report_name)

            # Create a folder for each report's results
            os.makedirs(output_folder, exist_ok=True)

            logging.info(f"Processing file: {report_file}")
            logging.info(f"Input Path: {input_path}")
            logging.info(f"Output Folder: {output_folder}\n")

            try:
                # Call the main workflow for each PDF
                main_workflow(input_path, output_folder)
                processed_files += 1
                logging.info(f"Finished processing: {report_file}\n")
            except Exception as e:
                errored_files.append(report_file)
                logging.error(f"Error processing file {report_file}: {e}\n")

        else:
            skipped_files += 1
            logging.warning(f"Skipping non-PDF file: {report_file}")

    # Time taken to process first 100 reports
    end_time = time.time()
    time_taken = end_time - start_time

    # Summary
    logging.info("PDF processing completed.")
    logging.info(f"Processed files: {processed_files}")
    logging.info(f"Skipped files: {skipped_files}")
    if errored_files:
        logging.error(f"Errored files: {len(errored_files)}")
        for file in errored_files:
            logging.error(f" - {file}")

    logging.info(f"Time taken to process first 100 reports: {time_taken:.2f} seconds.")
    logging.info("All tasks have been completed.\n")

In [26]:
if __name__ == "__main__":
    # Default folder paths
    reports_folder = "Reports"
    results_base_folder = "Results_Topic_Para"

    # Check if the reports folder exists
    if not os.path.exists(reports_folder):
        print(f"Error: Reports folder '{reports_folder}' does not exist.")
        sys.exit(1)

    # Call the processing function
    process_reports(reports_folder, results_base_folder)

2024-12-03 06:19:30,636 - INFO - Starting PDF processing from folder: Reports
2024-12-03 06:19:30,636 - INFO - Results will be saved in: Results_Topic_Para

2024-12-03 06:19:30,639 - INFO - Processing file: Technical-threat-report-Arid-Viper-April-2021.pdf
2024-12-03 06:19:30,642 - INFO - Input Path: Reports/Technical-threat-report-Arid-Viper-April-2021.pdf
2024-12-03 06:19:30,642 - INFO - Output Folder: Results_Topic_Para/Technical-threat-report-Arid-Viper-April-2021

2024-12-03 06:19:30,643 - INFO - Processing Reports/Technical-threat-report-Arid-Viper-April-2021.pdf and saving results in Results_Topic_Para/Technical-threat-report-Arid-Viper-April-2021
2024-12-03 06:19:30,643 - INFO - Step 0: Extracting text from PDF...
2024-12-03 06:19:30,699 - INFO - Step 0: Cleaning the extracted text...
2024-12-03 06:19:30,706 - INFO - Step 1: Chunking the cleaned text into sentences...
2024-12-03 06:19:31,758 - INFO - Step 2: Building the topic model...


2024-12-03 06:19:32,214 - BERTopic - Embedding - Transforming documents to embeddings.


2024-12-03 06:19:32,217 - INFO - Use pytorch device_name: cuda
2024-12-03 06:19:32,217 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

2024-12-03 06:19:35,788 - BERTopic - Embedding - Completed ✓
2024-12-03 06:19:35,789 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-12-03 06:19:40,298 - BERTopic - Dimensionality - Completed ✓
2024-12-03 06:19:40,299 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-12-03 06:19:40,321 - BERTopic - Cluster - Completed ✓
2024-12-03 06:19:40,324 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-12-03 06:19:45,766 - BERTopic - Representation - Completed ✓


...

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

2024-12-03 06:47:34,579 - BERTopic - Embedding - Completed ✓
2024-12-03 06:47:34,585 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-12-03 06:47:34,766 - BERTopic - Dimensionality - Completed ✓
2024-12-03 06:47:34,767 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-12-03 06:47:34,784 - BERTopic - Cluster - Completed ✓
2024-12-03 06:47:34,787 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-12-03 06:47:38,746 - BERTopic - Representation - Completed ✓


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

2024-12-03 06:47:38,998 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2024-12-03 06:47:38,999 - BERTopic - Dimensionality - Completed ✓
2024-12-03 06:47:39,000 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2024-12-03 06:47:39,005 - BERTopic - Cluster - Completed ✓


2024-12-03 06:47:39,005 - INFO - Step 3: Fetching representative documents for each topic...
2024-12-03 06:47:39,006 - INFO - Step 4: Processing topics for IoC extraction...
2024-12-03 06:47:39,091 - INFO - Step 5: Generating reports...
2024-12-03 06:47:39,091 - INFO - Processing chunk 1/1 of topic -1...
2024-12-03 06:47:39,112 - INFO - Saved STIX file: Results_Topic_Para/Weaponizing a Lazarus Group Implant/topic_-1/chunk_1/prediction_1.json
2024-12-03 06:47:39,118 - INFO - Saved STIX file: Results_Topic_Para/Weaponizing a Lazarus Group Implant/topic_-1/chunk_1/prediction_2.json
2024-12-03 06:47:39,118 - INFO - Processing chunk 1/1 of topic 0...
2024-12-03 06:47:39,216 - INFO - Saved STIX file: Results_Topic_Para/Weaponizing a Lazarus Group Implant/topic_0/chunk_1/prediction_1.json
2024-12-03 06:47:39,217 - INFO - Processing chunk 1/1 of topic 1...
2024-12-03 06:47:39,231 - INFO - Saved STIX file: Results_Topic_Para/Weaponizing a Lazarus Group Implant/topic_1/chunk_1/prediction_1.json


# Finish